In [ ]:
!pip install detecto

In [ ]:
import os
import torch
from detecto import core, utils, visualize
from detecto.visualize import show_labeled_image, plot_prediction_grid
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
custom_transforms = transforms.Compose([
transforms.ToPILImage(),
transforms.ToTensor(),
utils.normalize_transform(),
])

In [ ]:
# https://universe.roboflow.com/mail-vanbergeijk/frc2023-935
# https://universe.roboflow.com/michael-jansen/frc-charged-up-game-pieces/dataset/7
# https://universe.roboflow.com/fear1/chargedup-2023/dataset/3
# https://universe.roboflow.com/rawpower9-g6akz/cube-cube-det/dataset/6

# By far, the repo with the robots is the best one
!curl -L "https://universe.roboflow.com/ds/APxN012PDp?key=zfOj8HoVPe" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
cwd = './'

In [ ]:
Train_dataset=core.Dataset(cwd+'train',transform=custom_transforms)
Test_dataset=core.Dataset(cwd+'valid',transform=custom_transforms)
print("Size of Training Set: " + str(len(Train_dataset)))
print("Size of Testing Set: " + str(len(Test_dataset)))
Train_dataset[0][0].size()

In [ ]:
training_loader=core.DataLoader(Train_dataset, batch_size=32, shuffle=True)
validation_loader=core.DataLoader(Test_dataset, batch_size=32, shuffle=True)

In [ ]:
from detecto.core import Model
model_base = core.Model(['cone', 'cube', 'robot'], model_name= "fasterrcnn_mobilenet_v3_large_320_fpn")
# model_base = core.Model(['cone', 'cube', 'robot'])
model_base.get_internal_model().load_state_dict(torch.load('/content/model_cone_cube_50_epoch_60_percent.pth', map_location=model_base._device))

In [ ]:
# Low learning rate is needed, trained 10 epochs each while increasing learning_rate
losses = model_base.fit(training_loader, Test_dataset, epochs=50, lr_step_size=10, learning_rate=0.0000024, verbose=True)

In [ ]:
model_base.save('model_bot_weights.pth')

In [ ]:
plt.plot(losses)
plt.show()

In [ ]:
# image = utils.read_image('/content/test/PC060070_JPG.rf.c9c5ff945741674593f4ce3ec5f3bea6.jpg')
image = utils.read_image('/content/shadedmodel.png')
from detecto.core import Model
import cv2
import numpy as np

# image = cv2.resize(image, dsize=(320, 320), interpolation=cv2.INTER_CUBIC)
predictions = model_base.predict(image)
labels, boxes, scores = predictions
thresh=0.3
filtered_indices=np.where(scores>thresh)
filtered_scores=scores[filtered_indices]
filtered_boxes=boxes[filtered_indices]
num_list = filtered_indices[0].tolist()
filtered_labels = [labels[i] for i in num_list]
show_labeled_image(image, filtered_boxes, filtered_labels)